# Scraping Foundationcenter donation data


In [ ]:
from selenium.common.exceptions import NoSuchElementException, ElementClickInterceptedException
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
import time
import pandas as pd

url = r"https://maps.foundationcenter.org/#/list/?subjects=all&popgroups=all&years=all&location=6295630&excludeLocation=0&geoScale=ADM0&layer=geo_area&boundingBox=-183.1640625,-59.88893689676582,194.765625,79.30263962053658&gmOrgs=all&recipOrgs=all&tags=all&keywords=&pathwaysOrg=&pathwaysType=&acct=coronavirus&typesOfSupport=all&transactionTypes=all&amtRanges=all&minGrantAmt=0&maxGrantAmt=0&gmTypes=all&minAssetsAmt=0&maxAssetsAmt=0&minGivingAmt=0&maxGivingAmt=0&andOr=0&includeGov=1&custom=all&customArea=all&indicator=&dataSource=oecd&chartType=facets&multiSubject=1&listType=gm&windRoseAnd=undefined&zoom=2"

# driver = webdriver.Chrome(ChromeDriverManager().install())

driver = webdriver.Chrome(r"C:\Users\user\.wdm\drivers\chromedriver\83.0.4103.39\win32\chromedriver.exe")

driver.get(url)
time.sleep(5)

time.sleep(.1)

try:
    driver.find_element_by_xpath('//button[@class="close"]').click() #clicking to the X.
    print(' x out worked')
except NoSuchElementException:
    print(' x out failed')
    pass

data = []

all_grant = driver.find_elements_by_xpath("//tr[@class='gradeX odd' or  @class='gradeX odd' or @class='gradeX even' or  @class='even gradeX']")

number = 0
new_page = ""
total =  0


while new_page != "8":

    if count+1 >= len(all_grant):
        print("\nAbout to click next page")
        page = ['2','3','4','5','»','7','8'] # this is a the list of the pages to click
        new_page = page[number] 
        page_to_click= f'//*[@id="fm-list-1gmfm-list-table_wrapper"]//a[text()="{new_page}"]'
        next_page = driver.find_element_by_xpath(page_to_click).click() # click on next page
        time.sleep(5) # wait and let it load

        # make get the data on the page and pass it to all_grant
        all_grant = driver.find_elements_by_xpath("//tr[@class='gradeX odd' or  @class='gradeX odd' or @class='gradeX even' or  @class='even gradeX']")
        number += 1

        print(f"\nNow on page {number+1}")

    count = 1
    total = total + len(all_grant) # add previous data row to know the expected data row in total
    print("\nRunning new page for loop")
    for grant_row in all_grant:
        """ USING try and catch is optional but it is an important since we are not such what all the data looks like,
            and we want to keep our program running with been interrupted
        """
        print(f"Process {len(data)}/{total}")

        try:
            funder_name = grant_row.find_element_by_xpath('.//td[@class="cell-org-name header-org-foundation header"]').text
        except NoSuchElementException:
            funder_name = -1
            pass

        try:
            city = grant_row.find_element_by_xpath('.//td[@class="cell-city header"]').text
        except NoSuchElementException:
            city = -1
            pass
        
        try:
            state = grant_row.find_element_by_xpath('.//td[@class="cell-state header"]').text
        except NoSuchElementException:
            state = -1
            pass
        
        try:
            country = grant_row.find_element_by_xpath('.//td[@class="cell-country header"]').text
        except NoSuchElementException:
            country = -1
            pass
        
        try:
            value_granted = grant_row.find_element_by_xpath('.//td[@class="sorting_1 cell-dollar-value headerSortUp"]').text
        except NoSuchElementException:
            value_granted = -1
            pass
        
        try:
            no_of_grants = grant_row.find_element_by_xpath('.//td[@class="sorting_2 cell-nr-grants headerSortUp"]').text
        except NoSuchElementException:
            no_of_grants = -1
            pass

        if funder_name == -1 or country == -1 or grant == -1:
            count += 1
            continue

        row = {
                'funder_name' : funder_name,
                'city' : city,
                'state' : state,
                'country' : country,
                'value_granted' : value_granted,
                'no_of_grants' : no_of_grants
            }

        data.append(row)
        count += 1



In [4]:
df = pd.DataFrame(data)
df.to_csv("covid_donors.csv", index=False)

In [ ]:
['2','3','4','5','»','7','8']